# Budget Optimization

A company has 4 channels for advertizing

Channels: 
   - Search
   - Display
   - Youtube
   - Gmail

Variables: Money spend on 4 channels $X_i$, $i=1,2,3,4$

Contraints: 
- Search + Display + Youtube + Gmail $<=$ TotalBudget
- Display $>=$ 1000
- Youtube $>=$ 200
- Display + Search $>=$ 0.6 * TotalBudget
- Youtube $<=$ Gmail
- Gmail $<=$ 0.1 * TotalBudget

Minimum conversions (The left hand side can be predicted)

$sum_{i=1,2,3,4} X_i$ * channel_est_conversion_rate $>=$ MINCONVERSIONS


In [1]:
TOTALBUDGET = 5000
MINCONVERSIONS =  1500

library(data.table)
library(readxl)
library(lpSolveAPI)

file_str ='data/Conversion.xlsx'
conversion = as.data.table(read_excel(path = file_str, sheet = 1))

#Past data
channels = data.table(channel = c('Search','Display',
                                  'Youtube','Gmail'),
                      Cost = c(3001.18, 1743.18,284.22, 143.82),
                      Conversions = c(sum(conversion$Search),
                                      sum(conversion$Display),
                                      sum(conversion$YouTube),
                                      sum(conversion$Gmail)),
                      Revenue = c(69828,38720,7384,3519))

channels[,ConvCostRate := Conversions / Cost]
channels[, ROI := (Revenue - Cost) / Cost]

In [2]:
# Budget allocation optimization
nVar = 4
nCtr = 7

lprec <- make.lp(nCtr,nVar)


#Build variables
search_col_idx = 1
display_col_idx = 2
youtube_col_idx = 3
gmail_col_idx = 4

obj = as.matrix(channels$ROI)

row_lb = matrix(data = NA, nrow = nCtr)
row_ub = matrix(data = NA, nrow = nCtr)

#Build constraints
#Total Budget
row_ub[1] = TOTALBUDGET
set.row(lprec,1, rep(1,nVar),
        indices =  1:nVar)

#Display
row_ub[2] = -1000
set.row(lprec,2, -1,
        indices = display_col_idx)
#Youtube
row_ub[3] = -200
set.row(lprec,3, -1,
        indices = youtube_col_idx)

#Display + Search
row_ub[4] = - 0.6*TOTALBUDGET
set.row(lprec,4, c(-1,-1),
        indices = c(display_col_idx, search_col_idx))

#Youtube - Gmail
row_ub[5] = 0
set.row(lprec,5, c(1,-1),
        indices = c(youtube_col_idx, gmail_col_idx))

#Gmail
row_ub[6] = 0.1*TOTALBUDGET
set.row(lprec,6, 1,
        indices = gmail_col_idx)

#Total conversions
row_ub[7] = - MINCONVERSIONS
set.row(lprec,7, -channels$ConvCostRate,
        indices = 1:nVar)




#set lhs and rhs
set.constr.value(lprec, lhs = row_lb, rhs = row_ub, 1:nCtr)


# set objective function coefficients
set.objfn(lprec, obj)

#Controls
lp.control(lprec, sense = "max")


$anti.degen
[1] "fixedvars" "stalling" 

$basis.crash
[1] "none"

$bb.depthlimit
[1] -50

$bb.floorfirst
[1] "automatic"

$bb.rule
[1] "pseudononint" "greedy"       "dynamic"      "rcostfixing" 

$break.at.first
[1] FALSE

$break.at.value
[1] 1e+30

$epsilon
      epsb       epsd      epsel     epsint epsperturb   epspivot 
     1e-10      1e-09      1e-12      1e-07      1e-05      2e-07 

$improve
[1] "dualfeas" "thetagap"

$infinite
[1] 1e+30

$maxpivot
[1] 250

$mip.gap
absolute relative 
   1e-11    1e-11 

$negrange
[1] -1e+06

$obj.in.basis
[1] TRUE

$pivoting
[1] "devex"    "adaptive"

$presolve
[1] "none"

$scalelimit
[1] 5

$scaling
[1] "geometric"   "equilibrate" "integers"   

$sense
[1] "maximize"

$simplextype
[1] "dual"   "primal"

$timeout
[1] 0

$verbose
[1] "neutral"

In [3]:
lprec

Model name: 
                        C1                C2                C3                C4             
Maximize    22.26684837296   21.212278709024   24.979874744928   23.468085106368             
R1                       1                 1                 1                 1  free   5000
R2                       0                -1                 0                 0  free  -1000
R3                       0                 0                -1                 0  free   -200
R4                      -1                -1                 0                 0  free  -3000
R5                       0                 0                 1                -1  free      0
R6                       0                 0                 0                 1  free    500
R7        -0.3372007010575   -0.277653483863  -0.3659137288015  -0.3198442497565  free  -1500
Kind                   Std               Std               Std               Std             
Type                  Real              Real   

In [4]:
# write model to model.lp
#write.lp(lprec, "BudgetOpt.lp", type = c("lp"))


#solve the model
solStatusCode =  solve.lpExtPtr(lprec)


sols <- get.variables(lprec)
minimizer <- get.objective(lprec)



In [5]:
cat("the baudget for Search, Display, Youtube, Gmail is", sols)

the baudget for Search, Display, Youtube, Gmail is 3000 1000 500 500